# Criptografia con el uso de redes Neuronales


En este cuaderno, veremos cómo construir una red neuronal recurrente y entrenarla para descifrar cadenas cifradas con un determinado cifrado.

Este ejercicio lo familiarizará con las técnicas de preprocesamiento y construcción de modelos que le serán útiles cuando comience a construir modelos más avanzados para traducción automática, resumen de texto y más.

## DataSet
El dataset que tengo consta de 10,000 frases encriptadas y la versión en texto plano de cada frase encriptada.

Comencemos cargando el conjunto de datos para familiarizarnos con él. 

In [4]:

!wget -c https://raw.githubusercontent.com/franciscoventurablancas/CIC_IPN/master/Patrones/textoPlano.txt
!wget -c https://raw.githubusercontent.com/franciscoventurablancas/CIC_IPN/master/Patrones/cifrado.txt

--2021-10-11 20:43:34--  https://raw.githubusercontent.com/franciscoventurablancas/CIC_IPN/master/Patrones/textoPlano.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2021-10-11 20:43:34--  https://raw.githubusercontent.com/franciscoventurablancas/CIC_IPN/master/Patrones/cifrado.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



Cargamos los archivos que vamos utilizar 1 archivo con el texto plano y 1 archivo con el teto cifrado

In [5]:

def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data.split('\n')

In [6]:
import os

codes = load_data('cifrado.txt')
plaintext = load_data('textoPlano.txt')

Ahora, los "códigos" y el "texto sin formato" son matrices y cada elemento es una frase. Las primeras cinco frases codificadas son: 

In [7]:
codes[:5]

['YMJ QNRJ NX MJW QJFXY QNPJI KWZNY , GZY YMJ GFSFSF NX RD QJFXY QNPJI .',
 'MJ XFB F TQI DJQQTB YWZHP .',
 'NSINF NX WFNSD IZWNSL OZSJ , FSI NY NX XTRJYNRJX BFWR NS STAJRGJW .',
 'YMFY HFY BFX RD RTXY QTAJI FSNRFQ .',
 'MJ INXQNPJX LWFUJKWZNY , QNRJX , FSI QJRTSX .']

Y sus versiones de texto sin ser encriptadas son: 

In [8]:
plaintext[:5]

['THE LIME IS HER LEAST LIKED FRUIT , BUT THE BANANA IS MY LEAST LIKED .',
 'HE SAW A OLD YELLOW TRUCK .',
 'INDIA IS RAINY DURING JUNE , AND IT IS SOMETIMES WARM IN NOVEMBER .',
 'THAT CAT WAS MY MOST LOVED ANIMAL .',
 'HE DISLIKES GRAPEFRUIT , LIMES , AND LEMONS .']

In [9]:
len(codes)

10000

In [10]:
len(plaintext)

10000

In [11]:
max([len(sentence) for sentence in codes])

101

## Descripción general del modelo: RNN a nivel de  carácter
El modelo que usaremos aquí es un RNN(Red neuronal recurrente) a nivel de carácteres ya que el cifrado parece funcionar en el nivel de carácter. En un escenario de traducción automática, una RNN a nivel de palabra es la opción más común.

Un RNN a nivel de carácter tomará como entrada un entero que se refiere a un carácter específico y dará salida a otro entero. Para que nuestro modelo funcione, necesitaremos preprocesar nuestro conjunto de datos en los siguientes pasos:
 1. Aislar cada carácter como un elemento de la matriz (en lugar de una frase completa o una palabra como elemento de la matriz)
 2. Tokenizar los caracteres para que podamos convertirlos de letras a números enteros y viceversa
 3. Rellenar las cadenas para que todas las entradas y salidas quepan en forma de matriz
 
Para visualizar este procesamiento, supongamos que nuestras secuencias de origen ("códigos" en este caso) o secuencias de destino ("texto sin formato" en este caso) se ven así (una lista de cadenas):

<img src = "https://github.com/LaurentVeyssier/Cracking_code_using_RNN_at-character_level/blob/main/list_1.png?raw=1" />

Dado que este modelo funcionará en el nivel de carácter, necesitaremos separar cada cadena en una lista de caracteres (hecho implícitamente por el tokenizador en este cuaderno):

<img src = "https://github.com/LaurentVeyssier/Cracking_code_using_RNN_at-character_level/blob/main/list_2.png?raw=1" />

Luego, el proceso de tokenización convertirá cada carácter en un número entero. Tenga en cuenta que cuando está trabajando en un RNN a nivel de palabra (como en la mayoría de los ejemplos de traducción automática), el tokenizador asignará un número entero a cada palabra en lugar de a cada letra, y cada celda representaría una palabra en lugar de un carácter.

<img src = "https://github.com/LaurentVeyssier/Cracking_code_using_RNN_at-character_level/blob/main/list_3.png?raw=1" />

La mayoría de las plataformas de aprendizaje automático esperan que la entrada sea una matriz en lugar de una lista de listas. Para convertir la entrada en una matriz, necesitamos encontrar el miembro más largo de la lista y rellenar todas las secuencias más cortas con 0. Suponiendo que 'uno y dos' es la secuencia más larga en este ejemplo, la matriz termina luciendo así:

<img src = "https://github.com/LaurentVeyssier/Cracking_code_using_RNN_at-character_level/blob/main/padded_list.png?raw=1" />
 
## Preprocesamiento 
Para que una red neuronal prediga sobre datos de texto, primero debe convertirse en datos que pueda comprender. Los datos de texto como "perro" son una secuencia de codificaciones de caracteres ASCII. Dado que una red neuronal es una serie de operaciones de multiplicación y suma, los datos de entrada deben ser números.

Podemos convertir cada carácter en un número o cada palabra en un número. Estos se denominan identificadores de caracteres y palabras, respectivamente. Los identificadores de caracteres se utilizan para modelos de nivel de caracteres que generan predicciones de texto para cada caracter. Un modelo de nivel de palabra utiliza identificadores de palabras que generan predicciones de texto para cada palabra. Los modelos de nivel de palabra tienden a aprender mejor.

Convierta cada oración en una secuencia de identificadores de palabras usando la función [`Tokenizer`] (https://keras.io/preprocessing/text/#tokenizer) de Keras. Ya que estamos trabajando en el nivel del caracteres, asegúrese de establecer el indicador `char_level` en el valor apropiado. Luego, coloque el tokenizador en x. 

In [12]:
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    
    tf.keras.preprocessing.text.Tokenizer(num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=" ",
    char_level=False,
    oov_token=None,
    document_count=0,
    **kwargs
    )
    """
    # TODO: Implement
    x_tk = Tokenizer(char_level=True)
    x_tk.fit_on_texts(x)                 # because input is text, not sequence (list of integer tokens)

    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{' ': 1, 'e': 2, 'o': 3, 't': 4, 'i': 5, 's': 6, 'h': 7, 'r': 8, 'y': 9, 'u': 10, 'c': 11, 'n': 12, 'a': 13, 'p': 14, '.': 15, 'q': 16, 'k': 17, 'b': 18, 'w': 19, 'f': 20, 'x': 21, 'j': 22, 'm': 23, 'v': 24, 'l': 25, 'z': 26, 'd': 27, 'g': 28, ',': 29}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [4, 7, 2, 1, 16, 10, 5, 11, 17, 1, 18, 8, 3, 19, 12, 1, 20, 3, 21, 1, 22, 10, 23, 14, 6, 1, 3, 24, 2, 8, 1, 4, 7, 2, 1, 25, 13, 26, 9, 1, 27, 3, 28, 1, 15]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [18, 9, 1, 22, 3, 24, 2, 1, 29, 1, 23, 9, 1, 16, 10, 5, 11, 17, 1, 6, 4, 10, 27, 9, 1, 3, 20, 1, 25, 2, 21, 5, 11, 3, 28, 8, 13, 14, 7, 9, 1, 19, 3, 12, 1, 13, 1, 14, 8, 5, 26, 2, 1, 15]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [4, 7, 5, 6, 1, 5, 6, 1, 13, 1, 6, 7, 3, 8, 4, 1, 6, 2, 12, 4, 2, 12, 11, 2, 1, 15]


### Padding (IMPLEMENTACIÓN)
Al agrupar la secuencia de ID de palabras, cada secuencia debe tener la misma longitud. Dado que las oraciones tienen una longitud dinámica, podemos agregar relleno al final de las secuencias para que tengan la misma longitud.

Asegúrese de que todas las secuencias de cifrado tengan la misma longitud y que todas las secuencias de texto sin formato tengan la misma longitud agregando relleno al ** final ** de cada secuencia usando [`pad_sequences`] de Keras (https://keras.io/preprocessing/ función secuencia / # pad_sequences). 

In [13]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    
    tf.keras.preprocessing.sequence.pad_sequences(
    sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre',
    value=0.0)
    """
    # TODO: Implement
    # Find the length of the longest string in the dataset.
    if length is None:
        length = max([len(sentence) for sentence in x])
    # Then, pass it to pad_sentences as the maxlen parameter
    
    return pad_sequences(x, maxlen=length, padding="post", truncating="post",)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 4  7  2  1 16 10  5 11 17  1 18  8  3 19 12  1 20  3 21  1 22 10 23 14
  6  1  3 24  2  8  1  4  7  2  1 25 13 26  9  1 27  3 28  1 15]
  Output: [ 4  7  2  1 16 10  5 11 17  1 18  8  3 19 12  1 20  3 21  1 22 10 23 14
  6  1  3 24  2  8  1  4  7  2  1 25 13 26  9  1 27  3 28  1 15  0  0  0
  0  0  0  0  0  0]
Sequence 2 in x
  Input:  [18  9  1 22  3 24  2  1 29  1 23  9  1 16 10  5 11 17  1  6  4 10 27  9
  1  3 20  1 25  2 21  5 11  3 28  8 13 14  7  9  1 19  3 12  1 13  1 14
  8  5 26  2  1 15]
  Output: [18  9  1 22  3 24  2  1 29  1 23  9  1 16 10  5 11 17  1  6  4 10 27  9
  1  3 20  1 25  2 21  5 11  3 28  8 13 14  7  9  1 19  3 12  1 13  1 14
  8  5 26  2  1 15]
Sequence 3 in x
  Input:  [ 4  7  5  6  1  5  6  1 13  1  6  7  3  8  4  1  6  2 12  4  2 12 11  2
  1 15]
  Output: [ 4  7  5  6  1  5  6  1 13  1  6  7  3  8  4  1  6  2 12  4  2 12 11  2
  1 15  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0]


### Pipeline de preproceso
Su enfoque para este proyecto es construir una arquitectura de red neuronal, por lo que no le pediremos que cree una canalización de preproceso. En su lugar, le proporcionamos la implementación de la función `preprocess`. 

In [14]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_code_sentences, preproc_plaintext_sentences, code_tokenizer, plaintext_tokenizer =\
    preprocess(codes, plaintext)

print('Data Preprocessed')

Data Preprocessed


In [15]:
preproc_code_sentences[0]

array([ 5, 14,  3,  1, 10,  2, 13,  3,  1,  2,  4,  1, 14,  3,  6,  1, 10,
        3,  8,  4,  5,  1, 10,  2, 25,  3, 11,  1, 20,  6,  9,  2,  5,  1,
       18,  1, 17,  9,  5,  1,  5, 14,  3,  1, 17,  8,  7,  8,  7,  8,  1,
        2,  4,  1, 13, 15,  1, 10,  3,  8,  4,  5,  1, 10,  2, 25,  3, 11,
        1, 19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [16]:
len(code_tokenizer.word_index)+1  # +1 for 0?

32

In [17]:
len(plaintext_tokenizer.word_index)+1

32

In [18]:
plaintext_tokenizer.word_index

{' ': 1,
 "'": 31,
 ',': 18,
 '.': 19,
 '?': 30,
 'a': 8,
 'b': 17,
 'c': 22,
 'd': 11,
 'e': 3,
 'f': 20,
 'g': 16,
 'h': 14,
 'i': 2,
 'j': 26,
 'k': 25,
 'l': 10,
 'm': 13,
 'n': 7,
 'o': 12,
 'p': 21,
 'q': 28,
 'r': 6,
 's': 4,
 't': 5,
 'u': 9,
 'v': 23,
 'w': 24,
 'x': 27,
 'y': 15,
 'z': 29}

De forma predeterminada, la salida de una capa RNN contiene un solo vector por muestra. Este vector es la salida de la celda RNN correspondiente al último paso de tiempo, que contiene información sobre la secuencia de entrada completa. La forma de esta salida es (batch_size, units) donde las unidades corresponden al argumento de las unidades pasado al constructor de la capa.

Una capa RNN también puede devolver la secuencia completa de salidas para cada muestra (un vector por paso de tiempo por muestra), si establece return_sequences = True. La forma de esta salida es (tamaño de lote, pasos de tiempo, unidades).

Además, una capa RNN puede devolver sus estados internos finales. Los estados devueltos se pueden utilizar para reanudar la ejecución de RNN más tarde o para inicializar otro RNN. Esta configuración se usa comúnmente en el modelo de secuencia a secuencia del codificador-decodificador, donde el estado final del codificador se usa como el estado inicial del decodificador.

Para configurar una capa RNN para que devuelva su estado interno, establezca el parámetro return_state en True al crear la capa. Tenga en cuenta que LSTM tiene 2 tensores de estado, pero GRU solo tiene uno.

Para configurar el estado inicial de la capa, simplemente llame a la capa con el argumento de palabra clave adicional initial_state. Tenga en cuenta que la forma del estado debe coincidir con el tamaño de la unidad de la capa

retorna el estado:

- output, state_h, state_c = layers.LSTM(64, return_state=True, name="encoder")(x)
- encoder_state = [state_h, state_c]

In [19]:
preproc_code_sentences.shape

(10000, 101)

In [20]:
preproc_plaintext_sentences.shape

(10000, 101, 1)

# LSTM

In [21]:
from keras.layers import GRU, Input, Dense, TimeDistributed, RNN, LSTM
from keras.models import Model, Sequential
from keras.layers import Activation
from tensorflow.keras.optimizers import Adam # - Works
from keras.losses import sparse_categorical_crossentropy
import tensorflow


def simple_model(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param code_vocab_size: Number of unique code characters in the dataset
    :param plaintext_vocab_size: Number of unique plaintext characters in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the model
    
    x = Input(shape=input_shape[1:])   # shape(none,101,1) ie 
    
    seq = LSTM(units= 64, return_sequences = True, activation="tanh", name='Layer1')(x)  # output must be batchsize x timesteps x units
    
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)
    
    model = Model(inputs = x, outputs = output)
    
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    
    model.summary()
    
    return model


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_code_sentences, preproc_plaintext_sentences.shape[1]) # pad code sequences with maxlen 54: shape=10001x54
tmp_x = tmp_x.reshape((-1, preproc_plaintext_sentences.shape[-2], 1))     # reshape padded code seq in 10001 x 54 x 1


In [22]:
tmp_x.shape

(10000, 101, 1)

In [23]:
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 101, 1)]          0         
_________________________________________________________________
Layer1 (LSTM)                (None, 101, 64)           16896     
_________________________________________________________________
time_distributed (TimeDistri (None, 101, 32)           2080      
Total params: 18,976
Trainable params: 18,976
Non-trainable params: 0
_________________________________________________________________


In [24]:
simple_rnn_model.get_layer(name="Layer1").output.shape

TensorShape([None, 101, 64])

In [25]:
simple_rnn_model.fit(tmp_x, preproc_plaintext_sentences, batch_size=32, epochs=4, validation_split=0.2)

Epoch 1/4
250/250 [==============================] - 14s 29ms/step - loss: 1.5790 - accuracy: 0.5458 - val_loss: 1.0690 - val_accuracy: 0.6770
Epoch 2/4
250/250 [==============================] - 7s 27ms/step - loss: 0.8815 - accuracy: 0.7421 - val_loss: 0.7319 - val_accuracy: 0.7969
Epoch 3/4
250/250 [==============================] - 7s 26ms/step - loss: 0.6180 - accuracy: 0.8380 - val_loss: 0.5228 - val_accuracy: 0.8679
Epoch 4/4
250/250 [==============================] - 7s 26ms/step - loss: 0.4498 - accuracy: 0.8901 - val_loss: 0.3875 - val_accuracy: 0.9090


In [26]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], plaintext_tokenizer))

`logits_to_text` function loaded.
h h e   l i m e   i s   m e r   l e a s t   l i f e d   . r u i t   ,   b u t   t h e   m a n a n a   i s   m o   l e a s t   l i f e d   . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [27]:
plaintext[0]

'THE LIME IS HER LEAST LIKED FRUIT , BUT THE BANANA IS MY LEAST LIKED .'

#GRU (Gated Recurrent Unit) 
tiene como objetivo resolver el problema del 
gradiente de desaparición que viene con una red neuronal recurrente estándar.

In [28]:
def simple_model1(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param code_vocab_size: Number of unique code characters in the dataset
    :param plaintext_vocab_size: Number of unique plaintext characters in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the model
    
    x = Input(shape=input_shape[1:])   # shape(none,54,1) ie 
    
    seq = GRU(units= 64, return_sequences = True, activation="tanh", name='Layer1')(x)  # output must be batchsize x timesteps x units
    
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)
    
    model = Model(inputs = x, outputs = output)
    
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    
    model.summary()
    
    return model


In [29]:
# Train the neural network
simple_rnn_model1 = simple_model1(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 101, 1)]          0         
_________________________________________________________________
Layer1 (GRU)                 (None, 101, 64)           12864     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 101, 32)           2080      
Total params: 14,944
Trainable params: 14,944
Non-trainable params: 0
_________________________________________________________________


In [30]:
simple_rnn_model1.fit(tmp_x, preproc_plaintext_sentences, batch_size=128, epochs=16, validation_split=0.2)

Epoch 1/16
63/63 [==============================] - 4s 34ms/step - loss: 2.4952 - accuracy: 0.4382 - val_loss: 1.6317 - val_accuracy: 0.4725
Epoch 2/16
63/63 [==============================] - 2s 26ms/step - loss: 1.4500 - accuracy: 0.5365 - val_loss: 1.3084 - val_accuracy: 0.6051
Epoch 3/16
63/63 [==============================] - 2s 26ms/step - loss: 1.1915 - accuracy: 0.6505 - val_loss: 1.0918 - val_accuracy: 0.6783
Epoch 4/16
63/63 [==============================] - 2s 26ms/step - loss: 1.0005 - accuracy: 0.7129 - val_loss: 0.9230 - val_accuracy: 0.7366
Epoch 5/16
63/63 [==============================] - 2s 25ms/step - loss: 0.8502 - accuracy: 0.7531 - val_loss: 0.7886 - val_accuracy: 0.7731
Epoch 6/16
63/63 [==============================] - 2s 26ms/step - loss: 0.7289 - accuracy: 0.7985 - val_loss: 0.6793 - val_accuracy: 0.8133
Epoch 7/16
63/63 [==============================] - 2s 26ms/step - loss: 0.6294 - accuracy: 0.8330 - val_loss: 0.5882 - val_accuracy: 0.8502
Epoch 8/16
63

In [31]:
print(logits_to_text(simple_rnn_model1.predict(tmp_x[:1])[0], plaintext_tokenizer))

t h e   l i m e   i s   h e r   l e a s t   l i k e d   f r u i t   ,   b u t   t h e   b a n a n a   i s   m o   l e a s t   l i k e d   . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


# Una red neuronal recurrente (RNN)
Es una clase de redes neuronales artificiales donde las conexiones entre nodos forman un gráfico dirigido a lo largo de una secuencia temporal. 

In [32]:
from keras.layers import SimpleRNN 
def simple_model2(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param code_vocab_size: Number of unique code characters in the dataset
    :param plaintext_vocab_size: Number of unique plaintext characters in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the model
    
    x = Input(shape=input_shape[1:])   # shape(none,54,1) ie 
    
    seq = SimpleRNN(units= 64, return_sequences = True, activation="tanh", name='Layer1')(x)  # output must be batchsize x timesteps x units
    
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)
    
    model = Model(inputs = x, outputs = output)
    
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    
    model.summary()
    
    return model


In [33]:
# Train the neural network
simple_rnn_model2 = simple_model2(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 101, 1)]          0         
_________________________________________________________________
Layer1 (SimpleRNN)           (None, 101, 64)           4224      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 101, 32)           2080      
Total params: 6,304
Trainable params: 6,304
Non-trainable params: 0
_________________________________________________________________


In [ ]:
simple_rnn_model2.fit(tmp_x, preproc_plaintext_sentences, batch_size=128, epochs=16, validation_split=0.2)

Epoch 1/16
63/63 [==============================] - 9s 121ms/step - loss: 2.1609 - accuracy: 0.4174 - val_loss: 1.5379 - val_accuracy: 0.5428
Epoch 2/16
63/63 [==============================] - 7s 117ms/step - loss: 1.3808 - accuracy: 0.5904 - val_loss: 1.2605 - val_accuracy: 0.6204
Epoch 3/16
24/63 [==========>...................] - ETA: 4s - loss: 1.2126 - accuracy: 0.6356

In [ ]:
print(logits_to_text(simple_rnn_model2.predict(tmp_x[:1])[0], plaintext_tokenizer))

En general:
- el GRU(Gated Recurrent Unit) tiene la menor cantidad de errores (solo 1) pero requirió un mayor tiempo de entrenamiento con 16 épocas para 128 tamaños de lote
- SimpleRNN (recurrent neuronal networking) con el mismo entrenamiento tiene muchos más errores
- Finalmente, LSTM (Long short-term memory) tiene solo 2 errores y se entrenó con solo 4 épocas y 64 tamaños de lote, que es dos veces menor que el GRU

Conclusión: el mejor rendimiento para LSTM con GRU 